In [1]:
import sys
sys.path.append('../')
import newtable
import pprint

In [21]:
import copy
import dataclasses
@dataclasses.dataclass
class MyObj:
    name: str
    age: int
    height: float

    def asdict(self):
        return dataclasses.asdict(self)
    def asdict_nonone(self):
        return {k: v for k, v in dataclasses.asdict(self).items() if v is not None}

empty = {'name': None, 'age': None, 'height': None}
emptyobj = MyObj(**empty)


vals = list(range(1000))
vs1 = [{'age': i} for i in vals]
vs2 = [{'height': i, 'age': i, 'name': f'{i}_'} for i in vals]
def test_func0():
    vals = list()
    for act in vs1:
        vals.append(MyObj(**{k:act.get(k, empty[k]) for k,v in empty.items()}))
    return vals

def test_func01():
    vals = list()
    for act in vs1:
        new = copy.copy(emptyobj)
        for k,v in act.items():
            setattr(new, k, v)
        vals.append(new)
    return vals

def test_func1():
    return [MyObj(**{**empty, **v}) for v in vs1]
def test_func2():
    return [MyObj(**v) for v in vs2]
testobjs = [MyObj(**v) for v in vs2]
def test_func3():
    return [o.asdict_nonone() for o in testobjs]
def test_func4():
    return [o.asdict() for o in testobjs]
%timeit test_func0()
%timeit test_func01()
%timeit test_func1()
%timeit test_func2()
%timeit test_func3()
%timeit test_func4()

1.09 ms ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.91 ms ± 5.95 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
557 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
438 µs ± 538 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
7.57 ms ± 38.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.74 ms ± 5.35 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [2]:
core = newtable.ConnectCore.open(target=':memory:', dialect='sqlite')
core

ConnectCore(target=':memory:', dialect='sqlite', engine=Engine(sqlite:///:memory:), metadata=MetaData())

In [3]:
@newtable.table_schema
class MyContainer0:
    id: int = newtable.IDColumn()
    name: str
    age: int

with core.tablemaker() as tmaker:
    tab0 = tmaker.new_table(container_type=MyContainer0)
pprint.pprint(core.inspect_columns_all())

{'MyContainer0': [{'default': None,
                   'name': 'id',
                   'nullable': False,
                   'primary_key': 1,
                   'type': INTEGER()},
                  {'default': None,
                   'name': 'name',
                   'nullable': True,
                   'primary_key': 0,
                   'type': VARCHAR()},
                  {'default': None,
                   'name': 'age',
                   'nullable': True,
                   'primary_key': 0,
                   'type': INTEGER()}]}


In [13]:
import dataclasses
ignore_fields = ['id']
def new_dict(t: type):
    return {k: v for k,v in dataclasses.asdict(t).items() if k not in ignore_fields}

def inplace(t: type):
    d = dataclasses.asdict(t)
    for k in ignore_fields:
        del d[k]
    return d

print(dataclasses.is_dataclass(MyContainer0))
testobj = MyContainer0(id=0, name='test', age=10)
%timeit new_dict(testobj)
%timeit inplace(testobj)
%timeit dataclasses.asdict(testobj)

True
7.84 µs ± 447 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
6.87 µs ± 21.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
6.56 µs ± 12.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [4]:
with core.query() as q:
    q.insert_multi(tab0, [
        {'name': 'Devin J. Cornell', 'age': 50},
        {'name': 'Dorothy Andrews', 'age': 49},
    ])
    print(q.select(tab0).all())

[(1, 'Devin J. Cornell', 50), (2, 'Dorothy Andrews', 49)]


In [8]:
with tab0.query() as q:
    q.insert_single(MyContainer0(id=0, name='John Doe', age=30))
    print(q.select())

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: MyContainer0.id
[SQL: INSERT OR FAIL INTO "MyContainer0" (id, name, age) VALUES (?, ?, ?)]
[parameters: (0, 'John Doe', 30)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [ ]:
with core.connect() as conn:
    tab0 = newtable.DocTable.from_container(MyContainer0, core=core)
    conn.commit()
print(core.inspect_table_names())

InvalidRequestError: Table 'MyContainer0' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [ ]:
# create a new table in metadata (does not create table in database)
tab0 = newtable.DocTable.from_container(MyContainer0, core=core)
print(tab0)
print(core.inspect_table_names())

# creates a new doctable with the same table metadata
tab_tmp = newtable.DocTable.from_container(MyContainer0, core=core, extend_existing=True)
print(tab_tmp.table is tab0.table)
print(core.inspect_table_names())
core.connect().commit()
print(core.inspect_table_names())


InvalidRequestError: Table 'MyContainer0' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [ ]:
tab1 = core.tablemaker().new_table(container_type=MyContainer0, extend_existing=True)
pprint.pprint(tab1)

DocTable(schema=TableSchema(table_name='MyContainer0', container_type=<class '__main__.MyContainer0'>, columns=[ColumnInfo(attr_name='id', type_hint=<class 'int'>, params=ColumnParams(column_name=None, sqlalchemy_type=<class 'sqlalchemy.sql.sqltypes.Integer'>, foreign_key=None, type_kwargs=None, column_kwargs={'primary_key': True, 'autoincrement': True}, field_kwargs={})), ColumnInfo(attr_name='name', type_hint=<class 'str'>, params=ColumnParams(column_name=None, sqlalchemy_type=None, foreign_key=None, type_kwargs={}, column_kwargs={}, field_kwargs=None)), ColumnInfo(attr_name='age', type_hint=<class 'int'>, params=ColumnParams(column_name=None, sqlalchemy_type=None, foreign_key=None, type_kwargs={}, column_kwargs={}, field_kwargs=None))], indices=[], constraints=[], table_kwargs={}), table=Table('MyContainer0', MetaData(), Column('id', Integer(), table=<MyContainer0>, primary_key=True, nullable=False), Column('name', String(), table=<MyContainer0>), Column('age', Integer(), table=<MyC